In [ ]:
import gym
import gym_fish
import numpy as np
from stable_baselines3 import PPO, SAC
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')


gpuId = 0
env = gym.make('koi-cruising-v0', gpuId=gpuId)
# print(env.goal_pos, env.body_xyz)


model = PPO.load("koi-cruising-v0.zip", env=env)

print("model loaded")

steps = 0
obs = env.reset()

done_dist = 0.15
distance=k

x = np.random.uniform(5,5,size=(1,))
y=np.random.uniform(0,0,size=(1,))
z=np.random.uniform(5,5,size=(1,))
goal_pose_ = np.concatenate((x,y, z)).reshape((3, ))

env.goal_pos=goal_pose_

print("goal_pos is ",goal_pose_,"distnace is :  ",distance)

# Open the files in write mode
with open(f"{distance}_body_xyz_values.txt", "w") as xyz_file:
    error_history = []
    full_array = []

    while np.linalg.norm(env.goal_pos - env.body_xyz) > done_dist:
        action, states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        error = np.linalg.norm(env.goal_pos - env.body_xyz)
        print(f"output{steps}.png\terror: {error}")

        # Append the current error to the error history
        error_history.append(error)
        full_array.append(env.body_xyz)

        # Write env.body_xyz to the file
        xyz_file.write(f"{env.body_xyz}\n")

        # Ensure the error history doesn't grow indefinitely
        if len(error_history) > 20:
            error_history.pop(0)

        # Check if the error is increasing for the last 20 steps
        if len(error_history) == 20 and all(x < y for x, y in zip(error_history[:-1], error_history[1:])):
            print("Error has been increasing for the last 20 steps.")
            break

        steps += 1

